<a href="https://colab.research.google.com/github/richardOlson/nlp__tranformers/blob/main/pytorch_longtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working with pytorch to do the LongText classification

In [1]:
!pip install transformers

     |████████████████████████████████| 2.5 MB 6.6 MB/s 
     |████████████████████████████████| 895 kB 35.2 MB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 


In [2]:
# doing some imports 
import torch
import numpy as np
# getting the tokenizer and the model 
from transformers import  BertForSequenceClassification, AutoTokenizer

In [3]:
model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [5]:
# the text that is to be used that is longer than
# 512 when tokenized
text = """
I would like to get your all  thoughts on the bond yield increase this week.  I am not worried about the market downturn but the sudden increase in yields. On 2/16 the 10 year bonds yields increased by almost  9 percent and on 2/19 the yield increased by almost 5 percent.

Key Points from the CNBC Article:

* **The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.**
* **Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic.**
* **However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.**

The recent rise in bond yields and U.S. inflation expectations has some investors wary that a repeat of the 2013 “taper tantrum” could be on the horizon.

The benchmark U.S. 10-year Treasury note climbed above 1.3% for the first time since February 2020 earlier this week, while the 30-year bond also hit its highest level for a year. Yields move inversely to bond prices.

Yields tend to rise in lockstep with inflation expectations, which have reached their highest levels in a decade in the U.S., powered by increased prospects of a large fiscal stimulus package, progress on vaccine rollouts and pent-up consumer demand.

The “taper tantrum” in 2013 was a sudden spike in Treasury yields due to market panic after the Federal Reserve announced that it would begin tapering its quantitative easing program.

Major central banks around the world have cut interest rates to historic lows and launched unprecedented quantities of asset purchases in a bid to shore up the economy throughout the pandemic. The Fed and others have maintained supportive tones in recent policy meetings, vowing to keep financial conditions loose as the global economy looks to emerge from the Covid-19 pandemic.

However, the recent rise in yields suggests that some investors are starting to anticipate a tightening of policy sooner than anticipated to accommodate a potential rise in inflation.

With central bank support removed, bonds usually fall in price which sends yields higher. This can also spill over into stock markets as higher interest rates means more debt servicing for firms, causing traders to reassess the investing environment.

“The supportive stance from policymakers will likely remain in place until the vaccines have paved a way to some return to normality,” said Shane Balkham, chief investment officer at Beaufort Investment, in a research note this week.

“However, there will be a risk of another ‘taper tantrum’ similar to the one we witnessed in 2013, and this is our main focus for 2021,” Balkham projected, should policymakers begin to unwind this stimulus.

Long-term bond yields in Japan and Europe followed U.S. Treasurys higher toward the end of the week as bondholders shifted their portfolios.

“The fear is that these assets are priced to perfection when the ECB and Fed might eventually taper,” said Sebastien Galy, senior macro strategist at Nordea Asset Management, in a research note entitled “Little taper tantrum.”

“The odds of tapering are helped in the United States by better retail sales after four months of disappointment and the expectation of large issuance from the $1.9 trillion fiscal package.”

Galy suggested the Fed would likely extend the duration on its asset purchases, moderating the upward momentum in inflation.

“Equity markets have reacted negatively to higher yield as it offers an alternative to the dividend yield and a higher discount to long-term cash flows, making them focus more on medium-term growth such as cyclicals” he said. Cyclicals are stocks whose performance tends to align with economic cycles.

Galy expects this process to be more marked in the second half of the year when economic growth picks up, increasing the potential for tapering.

## Tapering in the U.S., but not Europe

Allianz CEO Oliver Bäte told CNBC on Friday that there was a geographical divergence in how the German insurer is thinking about the prospect of interest rate hikes.

“One is Europe, where we continue to have financial repression, where the ECB continues to buy up to the max in order to minimize spreads between the north and the south — the strong balance sheets and the weak ones — and at some point somebody will have to pay the price for that, but in the short term I don’t see any spike in interest rates,” Bäte said, adding that the situation is different stateside.

“Because of the massive programs that have happened, the stimulus that is happening, the dollar being the world’s reserve currency, there is clearly a trend to stoke inflation and it is going to come. Again, I don’t know when and how, but the interest rates have been steepening and they should be steepening further.”

## Rising yields a ‘normal feature’

However, not all analysts are convinced that the rise in bond yields is material for markets. In a note Friday, Barclays Head of European Equity Strategy Emmanuel Cau suggested that rising bond yields were overdue, as they had been lagging the improving macroeconomic outlook for the second half of 2021, and said they were a “normal feature” of economic recovery.

“With the key drivers of inflation pointing up, the prospect of even more fiscal stimulus in the U.S. and pent up demand propelled by high excess savings, it seems right for bond yields to catch-up with other more advanced reflation trades,” Cau said, adding that central banks remain “firmly on hold” given the balance of risks.

He argued that the steepening yield curve is “typical at the early stages of the cycle,” and that so long as vaccine rollouts are successful, growth continues to tick upward and central banks remain cautious, reflationary moves across asset classes look “justified” and equities should be able to withstand higher rates.

“Of course, after the strong move of the last few weeks, equities could mark a pause as many sectors that have rallied with yields look overbought, like commodities and banks,” Cau said.

“But at this stage, we think rising yields are more a confirmation of the equity bull market than a threat, so dips should continue to be bought.”
"""

In [6]:
# we are going to make it so that the tokenzier will return tensors
# of pytorch this time
tokens = tokenizer.encode_plus(text=text, add_special_tokens=False, return_tensors="pt")
print(tokens["input_ids"].shape)
tokens

Token indices sequence length is longer than the specified maximum sequence length for this model (1345 > 512). Running this sequence through the model will result in indexing errors


torch.Size([1, 1345])


{'input_ids': tensor([[1045, 2052, 2066,  ..., 4149, 1012, 1524]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
tokens["input_ids"][0]

tensor([1045, 2052, 2066,  ..., 4149, 1012, 1524])

In [ ]:
# using the split method
split_input_ids = torch.split(tokens["input_ids"][0], 510)

In [ ]:
len(split_input_ids)

3

In [ ]:
# my play around tensor
mTensor = torch.tensor([[1,2,3,4], [5,6,7,8]], dtype=torch.int64)
secondT = torch.tensor([[11, 12, 13, 14], [20, 21, 22, 23]], dtype=torch.int64)
print(mTensor, mTensor.shape)
print(secondT, secondT.shape)

tensor([[1, 2, 3, 4],
        [5, 6, 7, 8]]) torch.Size([2, 4])
tensor([[11, 12, 13, 14],
        [20, 21, 22, 23]]) torch.Size([2, 4])


In [ ]:
# making another of the tensors
l = torch.tensor([[1,2,3,4,5,6,7,8,9,10, 11, 12, 13 ,14 ,15, 16, 17, 18 , 19, 20]])
print(l.shape)
l

torch.Size([1, 20])


tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20]])

In [ ]:
# using the cat function to add the special tokens on the end and the beginning
cat = torch.cat((mTensor,secondT), dim=0)
cat

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [11, 12, 13, 14],
        [20, 21, 22, 23]])

In [ ]:
# using the split function 
c = torch.split(cat, 2)
c

(tensor([[1, 2, 3, 4],
         [5, 6, 7, 8]]), tensor([[11, 12, 13, 14],
         [20, 21, 22, 23]]))

In [ ]:
# splitting on just the inner tensor
split = torch.split(l[0], 4)
split

(tensor([1, 2, 3, 4]),
 tensor([5, 6, 7, 8]),
 tensor([ 9, 10, 11, 12]),
 tensor([13, 14, 15, 16]),
 tensor([17, 18, 19, 20]))

In [13]:
# will make the function that will create the slice of the tokens
def chunk_sentiment(t):
  # this is the amount not counting the special tokens on the ends.
  using_seq_len = 510 
  complete_chunk_size = 512
  # making the tensors that are going to be used to 
  # added on the front and the back of the chunks
  sep = torch.tensor(([tokenizer.sep_token_id]))
  cls = torch.tensor(([tokenizer.cls_token_id]))
  m = torch.tensor([1])

  print(f"This is the m {m}")
  print(f"This is the cls {cls}")
  print(f"This is the sep {sep}")

  if isinstance(t, str):
    # getting the tokens is passed in as a string
    tokens = tokenizer.encode_plus(t, add_special_tokens=False, return_tensors="pt")
  else:
    tokens = t
  print("Printing the tokens")
  print(tokens)
  # doing the splittng of the data
  # is a tuple of tensors
  # making them as lists so that they are not immutable
  input_ids_chunks = list(torch.split(tokens["input_ids"][0], using_seq_len))
  atten_mask_chunks = list(torch.split(tokens["attention_mask"][0], using_seq_len))

  # looping through the imput_ids chunck and the atten_mask_chunks
  for i in range(len(input_ids_chunks)):

  
    # adding the special tokens on the beginning and the end of the input_ids
    # and then adding the 1 on the begining and the end of attention mask
    input_ids_chunks[i] = torch.cat((cls, input_ids_chunks[i], sep), )
    atten_mask_chunks[i] = torch.cat((m, atten_mask_chunks[i], m), )

    # doing the padding of the input_id and the atten_mask
    if input_ids_chunks[i].shape[0] < complete_chunk_size:
      theLen = complete_chunk_size - input_ids_chunks[i].shape[0]
      input_ids_chunks[i] = torch.cat((input_ids_chunks[i], torch.tensor([tokenizer.pad_token_id] * theLen)))
      atten_mask_chunks[i] = torch.cat((atten_mask_chunks[i], torch.Tensor([tokenizer.pad_token_id] * theLen)))
  
  # now using stack to put all the input_chunks into the same tensor
  input_ids = torch.stack(input_ids_chunks, dim=0)
  attention_mask = torch.stack(atten_mask_chunks, dim=0)

  # now putting the input_ids and the attention mask into a dictionary
  input_dict = {"input_ids": input_ids.long(), "attention_mask": attention_mask.int()}

  # doing the sentiment
  probs = model(**input_dict)
  # doing the softmax to make the values of all add to 1
  probs_softmax = torch.nn.functional.softmax(probs[0], dim=-1)
  # finding the mean of all the tensors that are stacked
  mean_softmax = probs_softmax.mean(dim=0)
  # finding the argmax
  index = torch.argmax(mean_softmax).item()
  # returning as a tuple of the softmax and the index
  return mean_softmax, index




    



In [14]:
# trying out the function to see how it is working
chunk_sentiment(text)

This is the m tensor([1])
This is the cls tensor([101])
This is the sep tensor([102])
Printing the tokens
{'input_ids': tensor([[1045, 2052, 2066,  ..., 4149, 1012, 1524]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}


(tensor([0.4144, 0.4940, 0.0916], grad_fn=<MeanBackward1>), 1)